<a href="https://colab.research.google.com/github/dfridland/PyTorch/blob/HW6/PyTorch6Lesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

conv_1 = torch.nn.Conv1d(1, 1, kernel_size=5, bias=False)
conv_1.weight = torch.nn.Parameter(torch.FloatTensor([[[0.2, 0.2, 0.2, 0.2, 0.2]]]))

conv_1.weight, conv_1.bias

(Parameter containing:
 tensor([[[0.2000, 0.2000, 0.2000, 0.2000, 0.2000]]], requires_grad=True),
 None)

In [ ]:
tens = torch.FloatTensor([[93, 1655, 16, 288, 4, 163]])
conv_1(tens[None])

tensor([[[411.2000, 425.2000]]], grad_fn=<ConvolutionBackward0>)

In [ ]:
conv_1 = torch.nn.Conv1d(1, 2, kernel_size=5, bias=False)
conv_1.weight = torch.nn.Parameter(torch.FloatTensor([[[0.1, 0.1, 0.1, 0.1, 0.1],
                                                       [0.1, 0.1, 0.1, 0.1, 0.1]
                                                       ]]))

conv_1.weight, conv_1.bias

(Parameter containing:
 tensor([[[0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000, 0.1000, 0.1000]]], requires_grad=True),
 None)

In [ ]:
tens = torch.FloatTensor([[0.13, -0.11], [0.24, -0.19], [-0.01, -0.05], [0.07, -0.01],[0.01, 0.01]]).permute(1, 0)
conv_1(tens)

tensor([[0.0090]], grad_fn=<SqueezeBackward1>)

In [ ]:
tens.shape

torch.Size([2, 5])

In [ ]:
max_words = 2000
max_len = 20
num_classes = 1

#Training
epochs= 5
batch_size = 512
print_batch_n =  100


In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2023-04-12 09:47:25--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 108.177.126.138, 108.177.126.139, 108.177.126.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.126.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ad4jvbr85qiah8l9g9f9k2bm285btbqk/1681292775000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=300f0697-ec10-42ce-9388-3552d4d78ad9 [following]
--2023-04-12 09:47:29--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ad4jvbr85qiah8l9g9f9k2bm285btbqk/1681292775000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=300f0697-ec10-42ce-9388-3552d4d78ad9
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 108.177.119.132, 

In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: train.csv               
  inflating: val.csv                 


In [ ]:
import pandas as pd

df_train = pd.read_csv('train.csv')
df_val = pd.read_csv('val.csv')

In [ ]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [ ]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [ ]:
df_val.tail()

,id,text,class
22678,204145,А я знаю из-за кого такая паршивая погода была...,1
22679,204146,ааа.... что с Гмейлом. не ужто он умер сегодня...,0
22680,204147,"в учебники химии написано ""Пруст Жозеф Луи""\nя...",1
22681,204148,"Вот все любят,а я не люблю..как дура..говорю,ч...",0
22682,204149,уххх спасибо дорогой Леле Евгеньевне и Штепуху...,1


In [ ]:
df_val['class'].value_counts()

1    11449
0    11234
Name: class, dtype: int64

In [ ]:
#preprocessing

!pip install stop-words pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=23d153cf4b6f7f6a01cd30a2e1b037477d9afbbd603c7ce73b0197b53cad03aa
  Stored in directory: /root/.cache/pip/wheels/da/d8/66/395317506a23a9d1d7de433ad6a7d9e6e16aab48cf028a0f60
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=945e96186fd9beb1229be77caca7f44142d31c796389dad14f7e03c700244603
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built stop-words docopt


In [ ]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
sw = set(get_stop_words('ru'))
sw

{'а',
 'алло',
 'без',
 'близко',
 'более',
 'больше',
 'будем',
 'будет',
 'будете',
 'будешь',
 'будто',
 'буду',
 'будут',
 'будь',
 'бы',
 'бывает',
 'бывь',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'важная',
 'важное',
 'важные',
 'важный',
 'вам',
 'вами',
 'вас',
 'ваш',
 'ваша',
 'ваше',
 'ваши',
 'вверх',
 'вдали',
 'вдруг',
 'ведь',
 'везде',
 'весь',
 'вниз',
 'внизу',
 'во',
 'вокруг',
 'вон',
 'восемнадцатый',
 'восемнадцать',
 'восемь',
 'восьмой',
 'вот',
 'впрочем',
 'времени',
 'время',
 'все',
 'всегда',
 'всего',
 'всем',
 'всеми',
 'всему',
 'всех',
 'всею',
 'всю',
 'всюду',
 'вся',
 'всё',
 'второй',
 'вы',
 'г',
 'где',
 'говорил',
 'говорит',
 'год',
 'года',
 'году',
 'да',
 'давно',
 'даже',
 'далеко',
 'дальше',
 'даром',
 'два',
 'двадцатый',
 'двадцать',
 'две',
 'двенадцатый',
 'двенадцать',
 'двух',
 'девятнадцатый',
 'девятнадцать',
 'девятый',
 'девять',
 'действительно',
 'дел',
 'день',
 'десятый',
 'десять',
 'для',
 'до',
 'довольно',
 'до

In [ ]:
puncts = set(punctuation)
puncts

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~'}

In [ ]:
morpher = MorphAnalyzer()
morpher.parse('позавтракали')[0].normal_form

'позавтракать'

In [ ]:
morpher = MorphAnalyzer()

def preprocess_text(txt):
  txt = str(txt)
  txt = ''.join(c for c in txt if c not in puncts)
  txt = txt.lower()
  txt = re.sub('не\s', 'не', txt)
  txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
  return ' '.join(txt)

In [ ]:
df_train['text'].iloc[:1].values

array(['@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.'],
      dtype=object)

In [ ]:
df_train['text'].iloc[:1].apply(preprocess_text).values

array(['alisachachka неуезжаааааааать ❤ нехотеть уезжать'], dtype=object)

In [ ]:
from tqdm import tqdm
tqdm.pandas()

df_train['text'] = df_train['text'].progress_apply(preprocess_text)

df_val['text'] = df_val['text'].progress_apply(preprocess_text)

100%|██████████| 22683/22683 [00:28<00:00, 802.21it/s]


In [ ]:
train_corpus = ''.join(df_train['text'])
train_corpus = train_corpus.lower()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokens = word_tokenize(train_corpus)
tokens[:5]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['alisachachka', 'неуезжаааааааать', '❤', 'нехотеть', 'уезжатьrt']

In [ ]:
token_filtered = [word for word in tokens if word.isalnum()]

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(token_filtered)
token_filtered_top = [pair[0] for pair in  dist.most_common(max_words-1)]
len(token_filtered_top)

1999

In [ ]:
token_filtered_top[:10]

['хотеть',
 'd',
 'новый',
 'мой',
 'вообще',
 'завтра',
 'хороший',
 'день',
 'немочь',
 'любить']

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(token_filtered_top, 1)).items()}
vocabulary

{'хотеть': 1,
 'd': 2,
 'новый': 3,
 'мой': 4,
 'вообще': 5,
 'завтра': 6,
 'хороший': 7,
 'день': 8,
 'немочь': 9,
 'любить': 10,
 'делать': 11,
 'самый': 12,
 'думать': 13,
 'смотреть': 14,
 'идти': 15,
 'дом': 16,
 '2': 17,
 'один': 18,
 'друг': 19,
 'знать': 20,
 '3': 21,
 'такой': 22,
 'спать': 23,
 'утро': 24,
 'незнать': 25,
 'час': 26,
 'человек': 27,
 'школа': 28,
 'сидеть': 29,
 'сделать': 30,
 'небыть': 31,
 'писать': 32,
 'пойти': 33,
 'свой': 34,
 'ждать': 35,
 'любимый': 36,
 'блин': 37,
 'настроение': 38,
 'мама': 39,
 'никто': 40,
 'работа': 41,
 'нехотеть': 42,
 'ночь': 43,
 'написать': 44,
 'последний': 45,
 'говорить': 46,
 'какой': 47,
 'неделя': 48,
 'чтоть': 49,
 'болеть': 50,
 'мочь': 51,
 'фильм': 52,
 'посмотреть': 53,
 'первый': 54,
 'твой': 55,
 'добрый': 56,
 'урок': 57,
 'купить': 58,
 'снег': 59,
 'скоро': 60,
 'голова': 61,
 'пара': 62,
 'домой': 63,
 'равно': 64,
 'прийти': 65,
 'найти': 66,
 'дело': 67,
 'стать': 68,
 'давать': 69,
 'rt': 70,
 '5': 71,


In [ ]:
import numpy as np

def text_to_sequence(text, maxlen):
  result=[]
  tokens = word_tokenize(text.lower())
  tokens_filtered = [word for word in tokens if word.isalnum()]
  for word in tokens_filtered:
    if word in vocabulary:
      result.append(vocabulary[word])
  padding = [0] * (maxlen - len(result))
  return result[-maxlen:] + padding

In [ ]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train['text']])
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val['text']])


CPU times: user 22.4 s, sys: 164 ms, total: 22.5 s
Wall time: 26.3 s


In [ ]:
x_train.shape

(181467, 20)

In [ ]:
df_train['text'].iloc[0]

'alisachachka неуезжаааааааать ❤ нехотеть уезжать'

In [ ]:
x_train[0]

array([ 42, 628,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self, vocab_size=2000, embedding_dim=128, out_channel=128, num_classes=1):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.conv_1= nn.Conv1d(embedding_dim, out_channel, kernel_size=2)
    self.conv_2 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3)
    self.pool = nn.MaxPool1d(2)
    self.relu=nn.ReLU()
    self.linear_1 = nn.Linear(out_channel, out_channel // 2)
    self.linear_2 = nn.Linear(out_channel // 2, num_classes)

  def forward(self, x):
    output = self.embedding(x) 
    output = output.permute(0,2,1)

    output = self.conv_1(output)
    output = self.relu(output)
    output = self.pool(output)
    output = torch.max(output, axis=2).values
    output = self.linear_1(output)
    output = self.relu(output)
    output = self.linear_2(output)
    output = F.sigmoid(output)
    return output



In [ ]:
from torch.utils.data import DataLoader, Dataset

class DataWrapper(Dataset):
  def __init__(self, data, target, transform=None):
    self.data = torch.from_numpy(data).long()
    self.target = torch.from_numpy(target).long()
    self.transform = transform

  def __getitem__(self, index):
    x = self.data[index]
    y = self.target[index]

    if self.transform:
      x = self.transform(x)
    return x, y

  def __len__(self):
    return len(self.data)





In [ ]:
train_dataset = DataWrapper(x_train, df_train['class'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, df_val['class'].values)
val_loader =DataLoader(val_dataset, batch_size=8, shuffle=True)

In [ ]:
for x, l in train_loader:
  print(x.shape)
  print(l.shape)
  print(l[0])
  break

torch.Size([512, 20])
torch.Size([512])
tensor(0)


In [ ]:
model = Net(vocab_size=max_words)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

Net(
  (embedding): Embedding(2000, 128)
  (conv_1): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (linear_1): Linear(in_features=128, out_features=64, bias=True)
  (linear_2): Linear(in_features=64, out_features=1, bias=True)
)
Parameters: 346497


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
model = model.to(device)
model.train()
th=0.5

train_loss_history = []
test_loss_history = []

for epoch in range(epochs):
  running_items, running_right = 0.0, 0.0
  for i, data in enumerate (train_loader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()
    outputs = model(inputs)

    loss = criterion(outputs, labels.float().view(-1, 1))
    loss.backward()
    optimizer.step()

    loss = loss.item()
    running_items += len(labels)
    pred_labels = torch.squeeze((outputs > th).int())
    running_right += (labels == pred_labels).sum()

    model.eval()

    print(f'Epoch [{epoch + 1} / {epochs}]. ' \
          f'Step [{i + 1} / {len(train_loader)}]. ' \
          f'Loss: {loss:.3f}. ' \
          f'Acc: {running_right / running_items:.3f}', end='. ')
    
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
      test_labels = data[1].to(device)
      test_outputs = model(data[0].to(device))

      test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))

      test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
      test_running_total += len(data[1])
      pred_test_labels = torch.squeeze((test_outputs > th).int())
      test_running_right += (test_labels ++ pred_test_labels).sum()

  test_loss_history.append(test_loss.item())
  print(f'Test loss: {test_loss:.3f},' \
        f'Test acc:{test_running_right / test_running_total:.3f}'
   )

  model.train()
  print('Training is done!')



    
   

Epoch [1 / 5]. Step [1 / 355]. Loss: 0.693. Acc: 0.527. Epoch [1 / 5]. Step [2 / 355]. Loss: 0.910. Acc: 0.490. Epoch [1 / 5]. Step [3 / 355]. Loss: 0.760. Acc: 0.455. Epoch [1 / 5]. Step [4 / 355]. Loss: 0.735. Acc: 0.467. Epoch [1 / 5]. Step [5 / 355]. Loss: 0.694. Acc: 0.506. Epoch [1 / 5]. Step [6 / 355]. Loss: 0.684. Acc: 0.531. Epoch [1 / 5]. Step [7 / 355]. Loss: 0.702. Acc: 0.467. Epoch [1 / 5]. Step [8 / 355]. Loss: 0.690. Acc: 0.514. Epoch [1 / 5]. Step [9 / 355]. Loss: 0.673. Acc: 0.541. Epoch [1 / 5]. Step [10 / 355]. Loss: 0.671. Acc: 0.596. Epoch [1 / 5]. Step [11 / 355]. Loss: 0.661. Acc: 0.602. Epoch [1 / 5]. Step [12 / 355]. Loss: 0.660. Acc: 0.613. Epoch [1 / 5]. Step [13 / 355]. Loss: 0.654. Acc: 0.596. Epoch [1 / 5]. Step [14 / 355]. Loss: 0.648. Acc: 0.617. Epoch [1 / 5]. Step [15 / 355]. Loss: 0.655. Acc: 0.592. Epoch [1 / 5]. Step [16 / 355]. Loss: 0.660. Acc: 0.576. Epoch [1 / 5]. Step [17 / 355]. Loss: 0.648. Acc: 0.598. Epoch [1 / 5]. Step [18 / 355]. Loss: 0.

In [ ]:
import torch
conv_1 = torch.nn.Conv1d(2, 2, kernel_size=5, bias=False)
#conv_1.weight = torch.nn.Parameter(torch.FloatTensor([[
 #   [0.1, 0.1, 0.1, 0.1, 0.1],
 #   [0.1, 0.1, 0.1, 0.1, 0.1]
]]))
conv_1.weight, conv_1.bias